In [47]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Input, Embedding
from keras.models import Model
import preprocessdata as pe        ## It is builted module
import pandas as pd
import numpy as np
import string
from keras.models import load_model
# from sklearn.utils import shuffle
# import unicodedata
# import re
# punctuations = set(string.punctuation)

# digits_rm = str.maketrans('', '', string.digits)
# not_punc = ".?!,|"

# for c in not_punc:
#       punctuations.remove(c)

In [29]:
# def create_dataset(url, ):
#   df = pd.read_csv(url , encoding='utf-8')
#   df = df[df.columns[0:2]]
#   df = df[~pd.isnull(df['English'])]
#   df.drop_duplicates(inplace=True)
#   #df = df.sample(n=25000, random_state=42)
#   # Calculate number of words in each line of 2 columns
#   df['len_eng_words'] = df['English'].apply(lambda e: len(str(e).split(" ")))
#   df['len_hin_words'] = df['Hindi'].apply(lambda h: len(str(h).split(" ")))
#   df = df[df['len_eng_words']<=20]
#   df = df[df['len_hin_words']<=20]
  
#   return df

In [30]:
# def unicode_to_ascii(s):
#     return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

# def preprocessing_data_hin(w):
#     w = unicode_to_ascii(str(w).lower().strip())
#     w = re.sub(r"[\(\[].*?[\)\]]", ' ', w)
#     w = re.sub(r"([.?!,])", r' \1 ', w)
#     w = re.sub(r'["“”‘]', " ", w)
#     w = re.sub(r'[0-9]', " ", w)
    
#     w = w.strip()

#     #w = '<start> ' + w + ' <end>'
#     return w

# def preprocessing_data_eng(w):
#     w = unicode_to_ascii(str(w).lower().strip())
#     w = re.sub(r"[\(\[].*?[\)\]]", '', w)
#     w = re.sub(r"([.?!,])", r' \1 ', w)
#     w = re.sub(r'["]', " ", w)
#     w = re.sub(r'[^a-zA-Z?.!,]+', " ", w)

#     w = w.strip()

#     return w

In [31]:
# def customize_dataset(df):
#     df['Hindi'] = df['Hindi'].apply(lambda h : preprocessing_data_hin(h))
#     df['English'] = df['English'].apply(lambda e : preprocessing_data_eng(e))
#     df['Hindi'] = df['Hindi'].apply(lambda h: ''.join(c for c in h if c not in punctuations))
#     df['English'] = df['English'].apply(lambda e: ''.join(c for c in e if c not in punctuations))
#     df['English']=df['English'].apply(lambda e: e.translate(digits_rm))
#     df['Hindi']=df['Hindi'].apply(lambda h: h.translate(digits_rm))
#     df['Hindi']=df['Hindi'].apply(lambda h: re.sub("[a-z२३०८१५७९४६]", '', h))
#     df['English'] = df['English'].apply(lambda e : '<start> ' + e + ' <end>')
#     df['Hindi'] = df['Hindi'].apply(lambda h : '<start> ' + h + ' <end>')

#     return df['English'], df['Hindi']

In [32]:
# def tokenizing(sen):
#     words = []
#     for s in sen:
#         for w in s.split():
#             if w not in words:
#                 words.append(w)
#     return words

In [33]:
# def load_dataset():
#     df = create_dataset('https://raw.githubusercontent.com/prismspeechproject/neural/master/implementation/English_hindi.csv')

#     input_lang, target_lang = customize_dataset(df)

#     eng_words = sorted(tokenizing(input_lang))
#     hin_words = sorted(tokenizing(target_lang))

#     return input_lang, target_lang, eng_words, hin_words, df

In [34]:
# input_lang, target_lang, eng_words, hin_words, df = load_dataset()

# num_encoder_words, num_decoder_words, max_input_sen_len, max_target_sen_len = len(eng_words) + 1, len(hin_words) + 1, max(df['len_eng_words']), max(df['len_hin_words'])
'''
        Here I loaded imported prepocessdata.load_dataset
'''


url = 'https://raw.githubusercontent.com/prismspeechproject/neural/master/implementation/English_hindi.csv'
input_dict_token, target_dict_token, reverse_input_dict_token, reverse_target_dict_token, df = pe.load_dataset(url)
num_encoder_words  = len(input_dict_token) + 1
num_decoder_words = len(target_dict_token) + 1
max_input_sen_len = max(df['len_eng_words'])
max_target_sen_len = max(df['len_hin_words'])

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (7,8,9,10,11,12,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [35]:
# input_dict_token = dict([(word, i+1) for i, word in enumerate(eng_words)])
# target_dict_token = dict([(word, i+1) for i, word in enumerate(hin_words)])
# reverse_input_dict_token = dict([(i, word) for i, word in enumerate(eng_words)])
# reverse_target_dict_token = dict([(i, word) for i, word in enumerate(hin_words)])
# df = shuffle(df)


In [48]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df['English'], df['Hindi'], test_size=0.2 , random_state=42)
X_train.shape, X_test.shape

((99605,), (24902,))

In [49]:
def generate_batch(X, Y, batch_size=1):

    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_input_sen_len), dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_target_sen_len), dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_target_sen_len, num_decoder_words), dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j + batch_size], Y[j: j+batch_size])):
                for t, w in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_dict_token[w]
                for t, w in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_dict_token[w]
                    if t > 0:
                        decoder_target_data[i, t-1, target_dict_token[w]] = 1
            yield([encoder_input_data, decoder_input_data], decoder_target_data)


In [50]:

latent_dim = 300
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_words, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [51]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_words, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_words, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


In [52]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [53]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start word.
    target_seq[0, 0] = target_dict_token['<start>']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_dict_token[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '<end>' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [54]:
model = load_model('/content/drive/MyDrive/Colab/CopyUntitled4/nmt_weights22Ep100.h5')

In [55]:
model.summary

<bound method Model.summary of <keras.engine.functional.Functional object at 0x7f26f6b22910>>

In [56]:
wordLen3 = pd.read_csv('https://raw.githubusercontent.com/prismspeechproject/neural/colab-engines/implementation/TestDataset/WordLen3.csv')
wordLen3 = shuffle(wordLen3)
wordLen3X , wordLen3Y = wordLen3['English'], wordLen3['Hindi']

In [57]:
test_gen = generate_batch(wordLen3X, wordLen3Y, batch_size=1)
k =-1

In [58]:
k+=1
(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', wordLen3X[k:k+1].values[0])
print('Actual Hindi Translation:', wordLen3Y[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: <start> year old children <end>
Actual Hindi Translation: > तमिल बोलन वाल , <
Predicted Hindi Translation:  विधिवतता कठार सरटिफिकशन निरनतर बालको अथॉरीटी उ


In [59]:
k+=1
(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', wordLen3X[k:k+1].values[0])
print('Actual Hindi Translation:', wordLen3Y[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: <start> of the system . <end>
Actual Hindi Translation: > जाचना चाहता था . <
Predicted Hindi Translation:  जामाडोबा विचितरता पनिसलर कठपतलील कठपतलील कठपतलील त


In [60]:
k+=1
(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', wordLen3X[k:k+1].values[0])
print('Actual Hindi Translation:', wordLen3Y[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: <start> at stanford involved . <end>
Actual Hindi Translation: > सटनफोरड क ऊषमपरवगिकी विभाग को शामिल किया . <
Predicted Hindi Translation:  तलाशगा सजञाय डिजाइन चशम शिकसत तलल गजा रियायत दलबदलवि


In [61]:
k+=1
(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', wordLen3X[k:k+1].values[0])
print('Actual Hindi Translation:', wordLen3Y[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: <start> struggled with school . <end>
Actual Hindi Translation: > उन सब को सकली पढाई म दिककत हई थी । <
Predicted Hindi Translation:  तलाशगा कीऋए सशलिषट भगनावशिषट इनहोन मिडिया करमयोग ब


In [62]:
from nltk.translate.bleu_score import corpus_bleu

In [63]:
print('BLEU-1: %f' % corpus_bleu([actual_output], [decoded_sentence], weights=(1.0, 0, 0, 0)))
print('BLEU-2: %f' % corpus_bleu([actual_output], [decoded_sentence], weights=(0.5, 0.5, 0, 0)))
print('BLEU-3: %f' % corpus_bleu([actual_output], [decoded_sentence], weights=(0.33, 0.33, 0.33, 0)))
print('BLEU-4: %f' % corpus_bleu([actual_output], [decoded_sentence], weights=(0.25, 0.25, 0.25, 0.25)))

BLEU-1: 0.000000
BLEU-2: 0.000000
BLEU-3: 0.000000
BLEU-4: 0.000000


In [64]:
print('Individual 1-gram: %f' % corpus_bleu([actual_output], [decoded_sentence], weights=(1, 0, 0, 0)))
print('Individual 2-gram: %f' % corpus_bleu([actual_output], [decoded_sentence], weights=(0, 1, 0, 0)))
print('Individual 3-gram: %f' % corpus_bleu([actual_output], [decoded_sentence], weights=(0, 0, 1, 0)))
print('Individual 4-gram: %f' % corpus_bleu([actual_output], [decoded_sentence], weights=(0, 0, 0, 1)))

Individual 1-gram: 0.000000
Individual 2-gram: 0.000000
Individual 3-gram: 0.000000
Individual 4-gram: 0.000000
